In [18]:
import sys
sys.path.append('..')

In [19]:
from copy import deepcopy
from random import shuffle, choice
from library.solution import Solution
from library.algorithms.hill_climbing import hill_climbing

In previous notebooks, we defined the `Solution` class as an **abstract class** with methods that must be implemented by subclasses, depending on the specific optimization problem. While the implementation of solutions depend on the problem, all solutions share common principles: they require a **representation**, a **fitness function**, and a method for **random initialization**.

By extending this class, we can define solution classes specific to different optimization problems. For example, we created the `IntBinSolution` class to represent solutions for the IntBin optimization problem.

We then applied the Hill Climbing algorithm to the IntBin problem by further extending `IntBinSolution` to implement the `get_neighbors()` method, which is essential for Hill Climbing algorithm. To do this, we created a new class, `IntBinHillClimbingSolution`.

Today, we'll use Hill Climbing to solve two new problems: the Traveling Salesperson Problem (TSP) and the Knapsack Problem (KS).

## Traveling Salesperson Problem

**Description:** The Traveling Salesperson Problem (TSP) is the challenge of finding the shortest possible route that starts in a given city, visits each of the remaining N-1 cities exactly once, and returns to the starting city.

**Search space:** All possible permutations of city visit orders, forming valid round-trip routes.

**Representation:** List of city indexes that compose the route

**Fitness function:** f(x) = Total distance traveled, computed as the sum of distances between consecutive cities in the route.

**Goal:** Minimize f(x).

Let's begin by implementing the `TSPSolution` class, which inherits from `Solution`. As a result, it must implement the `fitness()` and `random_initial_representation()` methods.

This class represents a solution to the Traveling Salesperson Problem (TSP) and does not include any implementation related to the optimization algorithm that will be used to solve it.

![TSP Solution](images/tsp-solution.png)

In [21]:
from random import shuffle

l=[1,2,3]

shuffle(l)
l

[1, 3, 2]

In [22]:
len(l)

3

In [23]:
from library.problems.data.tsp_data import distance_matrix

#add some data and functions to work with it
#we have the python file in the data folder CIFO-24-25\library\problems\data
class TSPSolution(Solution):
    def __init__(self, repr=None, distance_matrix=distance_matrix, starting_idx=0): #setting the distance matrix from tsp data as the default
        self.distance_matrix = distance_matrix #self to store as an attribute
        self.starting_idx= starting_idx 
        super().__init__(repr=repr) #call the constructor of the parent class
        
    def random_initial_representation(self):
        route = {self.starting_idx}
        remaining_city_idx =[idx for idx in range(len(self.distance_matrix)) if idx != self.starting_idx]
        shuffle(remaining_city_idx)
        route = route + remaining_city_idx
        route.append(self.starting_idx)
        return route
    
    #now we have to implement the fitness method 
    def fitness(self):
        total_distance = 0 
        for list_idx in range(0, len(self.repr)-1):
            first_city_ix= self.repr[list_idx]
            next_city_ix = self.repr[list_idx+1]
            distance = self.distance_matrix[first_city_ix][next_city_ix]
            total_distance += distance
            # total_distance += self.distance_matrix[self.repr[i]][self.repr[i+1]]
        return total_distance
#TODO: Implement TSPSolution class

Let's test it

In [24]:
solution = TSPSolution()

print('Random solution:', solution)
print('Fitness:', solution.fitness())

TypeError: unsupported operand type(s) for +: 'set' and 'list'

### Solving TSP with Hill Climbing

To use Hill Climbing to solve TSP we need to define a `TSPHillClimbingSolution` class that implements the `get_neighbors()` method. We also need to ensure that this function returns a list of solutions that also implement the `get_neighbors()` method, therefore, return a list of solutions of type `TSPHillClimbingSolution` too.

A TSP neighbor solution is obtained by swapping the positions of two consecutive cities in the route.

![TSP Hill Climbing Solution](images/tsp-hillclimbing-solution.png)

In [ ]:
class TSPSHillClimbingSolution(TSPSolution):
    def get_neighbors(self):
        neighbors = []
        for list_idx in range(1, len(self.repr) -2):
            neighbor_repr= deepcopy(self.repr)
            neighbor_repr[list_idx] = self.repr[list_idx+1]
            neighbor_repr[list_idx+1] = self.repr[list_idx]
            
            neighbor = TSPSHillClimbingSolution(repr=neighbor_repr, 
                                                distance_matrix=self.distance_matrix, 
                                                starting_idx=self.starting_idx)
            
            neighbors.append(neighbor)
        return neighbors
    
    
#in this case if we have n cities we have n-2 neighbors

Let's test it

In [ ]:
solution = TSPHillClimbingSolution()
print('Solution:', solution)

neighbors = solution.get_neighbors()
print('Neihghbors:')
for neighbor in neighbors:
    print(neighbor)

And now we can apply the hill climbing algorithm by passing it a random initial solution.

In [ ]:
#TODO: Apply hill climbing to TSP
initial_solution = TSPSHillClimbingSolution() #random solution with a random representation
hill_climbing(initial_solution, maximization=False, verbose=True)

The implementations of `TSPSolution` and `TSPHillClimbingSolution` can be found in `library/problems/tsp.py`

## Knapsack Problem

**Description:** The Knapsack Problem involves selecting a subset of N items, each with a given value and weight, to pack into a container with a fixed capacity. If the total weight of selected items exceeds the capacity, the solution is invalid. The goal is to maximize the total value of items while ensuring they fit within the container's constraints.

**Search space:** All possible subsets of items that can be placed in the knapsack.

**Representation:** Binary string of length N (number of items), where 1 indicates the item is included in the knapsack and 0 indicates the item is excluded.

**Fitness function:** f(x)= Total value inside the knapsack. If the total size exceeds the knapsack's capacity, the solution is invalid and assigned a fitness of -inf.

**Goal:** Maximize f(x).

Similarly to what we've done for TSP, let's begin by implementing the `KSSolution` class, which inherits from `Solution` and implementes the `fitness()` and `random_initial_representation()` methods.

This class represents a solution to the Knapsack problem (KS) and does not include any implementation related to the optimization algorithm that will be used to solve it.

In [25]:
from library.problems.data.ks_data import values, weights, capacity
from random import choice
# we can also set the fitness as- total_capacity 

#try to get out of the fitnes bad landscape (?)

#TODO: Implement KSSolution class
class KSSolution(Solution):
    def __init__(self, values, weights, capacity,  repr=None):
        self.values = values
        self.weights = weights
        self.capacity = capacity
        super().__init__(repr=repr) #saving as an attribute the values, weights and capacity
        
        #implementing the random initial representation
    def random_initial_representation(self):
        repr = [] 
        #now lets add a zero or a one 
        for item_idx in range(len(self.values)):
            repr.append(choice([0,1])) #randomly choose 0 or 1
        return repr
    
    def total_weight(self):
        total_weight = 0
        for item_idx in range(len(self.repr)):
            if self.repr[item_idx] == 1:
                total_weight += self.weights[item_idx]
        return total_weight
    
    def total_value(self):
        total_value = 0
        for item_idx in range(len(self.repr)):
            if self.repr[item_idx] == 1:
                total_value += self.values[item_idx]
        return total_value
    
    #total value and total weight are the helper fitness functions just to be more clear and more readable
    
    def fitness(self):
        total_weight = self.total_weight()
        if total_weight > self.capacity:
            return -999999999999999
        else:
            return self.total_value()

Let's test it

In [36]:
solution = KSSolution(values=values, weights=weights, capacity=capacity)

print(solution)
print(solution.fitness())

[1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0]
-999999999999999


### Solving KS with Hill Climbing

A neighbor solution is obtained by flipping a single bit, meaning adding one item to the knapsack, or removing one item from the knapsack.

Let's create the `KSHillClimbingSolution` that inherits from `KSSolution` and implements the `get_neighbors` method.


In [37]:
#TODO: Implement KSHillClimbingSolution class
def KSHillClimbingSolution(KSSolution):
    def get_neighbors(self):
        neighbors = []
        for item_idx in range(len(self.repr)):
            neighbor_repr = deepcopy(self.repr)
            if self.repr[item_idx] == 1:
                neighbor_repr[item_idx] = 0
            else:
                neighbor_repr[item_idx] = 1
            
            neighbor = KSHillClimbingSolution(values=self.values, weights=self.weights, capacity=self.capacity, repr=neighbor_repr)
            neighbors.append(neighbor)
        return neighbors    

Let's test it

In [38]:
solution = KSHillClimbingSolution(values=values, weights=weights, capacity=capacity)
print('Solution:', solution)

neighbors = solution.get_neighbors()
print('Neihghbors:')
for neighbor in neighbors:
    print(neighbor)

TypeError: KSHillClimbingSolution() got an unexpected keyword argument 'values'

And now we can apply the hill climbing algorithm by passing it a random initial solution.

In [ ]:
#TODO: Apply hill climbing to KS problem
initial_solution = KSHillClimbingSolution(values=values, weights=weights, capacity=capacity)
best_solution = hill_climbing(initial_solution, maximization=True, verbose=True)
print('Best solution:', best_solution)
print('Fitness:', best_solution.fitness())

The implementations of `KSSolution` and `KSHillClimbingSolution` can be found in `library/problems/knapsack.py`